## 4.2 NLP(ish)

https://maartengr.github.io/BERTopic/getting_started/quickstart/quickstart.html

## décider de comment on découpe
- sentence ?
- paragraphe (existe pas ici) -> mais possible avec wtpsplit : https://github.com/segment-any-text/wtpsplit
- dire que tant pis ?
- faire un check global du nb token par intervention et cut que ce qui dépasse ?
- etc.

In [1]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from sentence_transformers import SentenceTransformer
from sklearn.feature_extraction.text import CountVectorizer
import bertopic
from bertopic import BERTopic
from sklearn.feature_extraction.text import CountVectorizer
from sentence_transformers import SentenceTransformer
import spacy
from bertopic.vectorizers import ClassTfidfTransformer

In [2]:
df = pd.read_csv(
    "../data/interim/df_repu.csv", low_memory=False, dtype={"ID_orateur": str}
)

In [3]:
df["DateSeance_ts"] = pd.to_datetime(df["DateSeance"], format="%Y%m%d%H%M%S%f")
df["DateSeance_day"] = df["DateSeance_ts"].dt.normalize()  # guess it works


## Bert things

In [45]:
#########
# Si besoin, revenir au plus simple :
#########

# nltk.download("stopwords")
# french_stopwords = list(set(stopwords.words("french")))
# vectorizer_model = CountVectorizer(stop_words=french_stopwords)
# topic_model = bertopic.BERTopic(language="french", vectorizer_model=vectorizer_model)
# # topics, probs = topic_model.fit_transform(df["Texte_clean"])

In [4]:
# TODO: redo with the good sentencetransformer

Stopwords :
- https://maartengr.github.io/BERTopic/getting_started/tips_and_tricks/tips_and_tricks.html#document-length
- Instead, we can use the CountVectorizer to preprocess our documents after having generated embeddings and clustered our documents. Personally, I have found almost no disadvantages to using the CountVectorizer to remove stopwords and it is something I would strongly advise to try out:
- We can also use the ClassTfidfTransformer to reduce the impact of frequent words. The end result is very similar to explicitly removing stopwords but this process does this automatically:

In [ ]:
# # tester un truc autour de alibaba pour l'embedding ?

# model_name_or_path="Alibaba-NLP/gte-multilingual-base"

In [ ]:
# vectorizer_model et french stopwords
# Avec spacy
nlp = spacy.load("fr_core_news_sm")
french_stopwords = list(nlp.Defaults.stop_words)
vectorizer_model = CountVectorizer(stop_words=french_stopwords)

# # Ou avec nltk
# import nltk
# from nltk.corpus import stopwords
# nltk.download("stopwords")
# french_stopwords = list(set(stopwords.words("french")))
# vectorizer_model = CountVectorizer(stop_words=french_stopwords)

# "dangvantuan/sentence-camembert-large"
# "all-MiniLM-L6-v2"

# TODO: repacer au all mini parce qu'on pleure en GPU

embedding_model = SentenceTransformer(
   , trust_remote_code=True
embedding_model = SentenceTransformer(
    "Alibaba-NLP/gte-multilingual-base", trust_remote_code=True
)  # TODO: vérifier usage GPU (device= ?)

# ctfidf_model = ClassTfidfTransformer(reduce_frequent_words=True)

# créer le modèle
topic_model_better = BERTopic(
    embedding_model=embedding_model,
    vectorizer_model=vectorizer_model,  # remove stopwords after embbedings
    # ctfidf_model=ctfidf_model, # (or) reduce the impact of frequent word
)

configuration.py: 0.00B [00:00, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/Alibaba-NLP/new-impl:
- configuration.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling.py: 0.00B [00:00, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/Alibaba-NLP/new-impl:
- modeling.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


model.safetensors:   0%|          | 0.00/611M [00:00<?, ?B/s]

Some weights of the model checkpoint at Alibaba-NLP/gte-multilingual-base were not used when initializing NewModel: ['classifier.bias', 'classifier.weight']
- This IS expected if you are initializing NewModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing NewModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [8]:
# Fiter le modèle
topics_better, probs_better = topic_model_better.fit_transform(df["Texte_clean"])

RuntimeError: Invalid buffer size: 42.68 GB

In [23]:
# help(topic_model_better)

In [39]:
topic_model_better.get_topic_info()[:20]

,Topic,Count,Name,Representation,Representative_Docs
0,-1,1907,-1_de_la_le_et,"[de, la, le, et, les, des, que, en, nous, qui]",[Nous sommes une démocratie nucléaire et je cr...
1,0,326,0_lcole_de_lves_des,"[lcole, de, lves, des, les, et, la, en, le, pour]",[Je suis heureuse de vous retrouver autour du ...
2,1,181,1_rpublique_vous_la_cest,"[rpublique, vous, la, cest, pas, ve, que, dmoc...","[Ce n’est pas ça, la République !, Vous dépeig..."
3,2,106,2_des_de_violences_la,"[des, de, violences, la, et, les, le, en, qui,...","[…maire de Marcq-en-Barœul, dans ma circonscri..."
4,3,106,3_femmes_de_la_et,"[femmes, de, la, et, des, les, que, le, nous, ...",[Aujourd’hui est un grand jour : nous pouvons ...
5,4,64,4_de_des_les_et,"[de, des, les, et, la, territoires, en, le, da...",[Les territoires ultramarins sont des joyaux d...
6,5,47,5_vous_franais_rpublique_et,"[vous, franais, rpublique, et, je, ensemble, l...",[…mais encore au centre de notre pacte social ...
7,6,45,6_rpublicain_rpublicaine_lordre_cest,"[rpublicain, rpublicaine, lordre, cest, rpubli...","[Je ne pousse pas de cris d’orfraie, je demand..."
8,7,42,7_vous_la_de_que,"[vous, la, de, que, le, et, pour, pas, les, qui]","[« Sans ordre républicain, il n’y a ni sécurit..."
9,8,41,8_rpublique_droite_lextrme_la,"[rpublique, droite, lextrme, la, pas, et, vous...","[…tout en cherchant, en même temps, à négocier..."


In [40]:
# Optional: visualize
topic_model_better.visualize_topics().show()

In [ ]:
topic_model_better.save("../models/bert/modele_bert_better", save_embedding_model=True)

2025-06-30 00:25:23,641 - BERTopic - WARNING: When you use `pickle` to save/load a BERTopic model,please make sure that the environments in which you saveand load the model are **exactly** the same. The version of BERTopic,its dependencies, and python need to remain the same.


## Dynamic topic model

There are a few important parameters that you should take note of, namely:

* `docs`
  * the texts that we are using
* `timestamps`
  * The timestamp of each document
* `global_tuning`
  * Whether to average the topic representation of a topic at time *t* with its global topic representation
* `evolution_tuning`
  * Whether to average the topic representation of a topic at time *t* with the topic representation of that topic at time *t-1*
* `nr_bins`
  * The number of bins to put our timestamps into. It is computationally inefficient to extract the topics at thousands of different timestamps. Therefore, it is advised to keep this value below 20.


In [41]:
topics_over_time_better = topic_model_better.topics_over_time(
    docs=df["Texte_clean"],
    timestamps=df["DateSeance_ts"],
    global_tuning=True,
    evolution_tuning=True,
    nr_bins=20,
)

18it [00:02,  6.42it/s]


In [42]:
topic_model_better.visualize_topics_over_time(topics_over_time_better, top_n_topics=40)

## Hierarchical topics

In [43]:
hierarchical_topics = topic_model_better.hierarchical_topics(df["Texte_clean"])

100%|██████████| 46/46 [00:00<00:00, 536.62it/s]


In [47]:
topic_model_better.visualize_hierarchy(hierarchical_topics=hierarchical_topics)

## Topic reduction

In [48]:
topic_model_better.reduce_topics(df["Texte_clean"], nr_topics=20)

2025-07-02 18:46:30,812 - BERTopic - Topic reduction - Reducing number of topics
2025-07-02 18:46:30,813 - BERTopic - Topic reduction - Reduced number of topics from 20 to 20


voir le modèle dans le notebook emilien qui découpe en paragraphe sans avoir de marqueur de paragrphae. et garder que celui qui contient répu.

Passer à l'échelle de la phrase pour tout ce qui va être pour sentiment, réseau de mots, proba des termes, etc.